In [118]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Normalizer
import jieba
import re 
import json
comment_cut = pd.read_csv('../Top_frequenct_words/commentcut.csv')
with open('../Top_frequenct_words/geo_zh_en_dict.json') as f:
    geo_zh_en_dict = json.load(f)
with open('../Top_frequenct_words/top_freq_words.json') as f:
    top_freq_words = json.load(f)
geo_zh_en_dict['中国'] = 'China'

In [3]:
import shifterator as sh

In [15]:
import collections
def clean_text(doc_list):
    words = []
    for doc in doc_list:
        words += doc[1:-1].replace("'", "").split(", ")
    wordfreq_dict = collections.Counter(words)
    return wordfreq_dict

In [114]:
geo_jsd_dict = {}
geo_entropy_dict = {}

In [119]:
for geo_zh in geo_zh_en_dict:
    if geo_zh == '中国':
        geo_texts_non = comment_cut[(comment_cut['label']==0) & (comment_cut['location']!='海外')]['cut'].tolist()
        geo_texts_sex = comment_cut[(comment_cut['label']==1) & (comment_cut['location']!='海外')]['cut'].tolist()
    else:
        geo_texts_non = comment_cut[(comment_cut['label']==0) & (comment_cut['location']==geo_zh)]['cut'].tolist()
        geo_texts_sex = comment_cut[(comment_cut['label']==1) & (comment_cut['location']==geo_zh)]['cut'].tolist()
    geo_freq_non = clean_text(geo_texts_non)
    geo_freq_sex = clean_text(geo_texts_sex)
    jsd_shift = sh.JSDivergenceShift(type2freq_1=geo_freq_non,
                                     type2freq_2=geo_freq_sex,
                                     weight_1=0.5,
                                     weight_2=0.5,
                                     base=2,
                                     alpha=1)
    entropy_shift = sh.EntropyShift(type2freq_1=geo_freq_non,
                                    type2freq_2=geo_freq_sex,
                                    base=2)
    geo_jsd_dict[geo_zh] = jsd_shift
    geo_entropy_dict[geo_zh] = entropy_shift

In [120]:
geo_entropy_dict['中国'].get_shift_scores()

{'': -0.00011762167926838095,
 '男弱': 5.709237188836205e-06,
 '汤兰兰': -2.588409774066135e-05,
 '才艺': 5.709237188836205e-06,
 '捆绑': 0.00013420129528839375,
 '店员': 5.709237188836205e-06,
 '伺候': 0.00024343849400404008,
 '魔抓': -2.588409774066135e-05,
 'ISIS': -2.588409774066135e-05,
 '直男哥哥': 3.159333492949756e-05,
 '上价': 3.159333492949756e-05,
 '上次': -1.9906107692662997e-05,
 '童婚': -1.078170615307013e-05,
 '起源于': -4.8514123753465344e-05,
 '暴打': 3.159333492949756e-05,
 '标志': -6.991592072701785e-05,
 '20万': 5.9134214573947714e-05,
 '误导人': -2.588409774066135e-05,
 '厌世': -4.8514123753465344e-05,
 '磋磨': 3.159333492949756e-05,
 '主义们': -2.588409774066135e-05,
 '1%': 5.709237188836205e-06,
 '艺术': -0.0003720917428108134,
 '逃学': -2.588409774066135e-05,
 '代母': 3.159333492949756e-05,
 '解放前': -2.588409774066135e-05,
 '成熟点': -2.588409774066135e-05,
 '冤冤相报': 3.159333492949756e-05,
 '气好': -2.588409774066135e-05,
 '抗议性': -2.588409774066135e-05,
 '爱慕': 5.709237188836205e-06,
 '信号': -2.588409774066135e-05,
 '咆

In [121]:
res_list = []
for geo_zh in geo_zh_en_dict:
    geo_en = geo_zh_en_dict[geo_zh]
    res_geo_dict = {
        'region': geo_en,
    }
    geo_jsd = geo_jsd_dict[geo_zh]
    geo_entropy = geo_entropy_dict[geo_zh].get_shift_scores()
    jsd_word_dict_non = jsd_shift.type2p_1
    jsd_word_dict_sex = jsd_shift.type2p_2
    top_words_list = top_freq_words[geo_en]
    word_list = []
    for word in top_words_list:
        jsd_non = -1
        jsd_sex = -1
        word = re.findall("[\u4e00-\u9fa5]+", word)
        if len(word) == 0:
            continue
        word = word[0]
        if word not in geo_entropy:
            entropy = 0
        else:
            entropy = geo_entropy[word]
        if word in jsd_word_dict_non:
            jsd_non = jsd_word_dict_non[word]
        if word in jsd_word_dict_sex:
            jsd_sex = jsd_word_dict_sex[word]
        if jsd_non == -1 and jsd_sex == -1:
            continue
        if jsd_non >= jsd_sex:
            word_list.append({"word": word, "label": 0, "value": jsd_non, "entropy" :abs(entropy)})
        else:
            word_list.append({"word": word, "label": 1, "value": jsd_sex, "entropy" :abs(entropy)})
    jsd_val_list = [item['value'] for item in word_list]
    entropy_val_list = [item['entropy'] for item in word_list]
    jsd_val_list_norm = Normalizer(norm='max').fit_transform([jsd_val_list])[0]
    entropy_val_list_norm = Normalizer(norm='max').fit_transform([entropy_val_list])[0]
    for i in range(len(word_list)):
        word_list[i]['value'] = jsd_val_list_norm[i]
        word_list[i]['entropy'] = entropy_val_list_norm[i]
    word_list = sorted(word_list, key=lambda d: d['value'], reverse=True) 
    res_geo_dict['word_list'] = word_list
    res_list.append(res_geo_dict)

In [122]:
with open('./word_divergence.json', 'w') as f:
    json.dump(res_list, f)

In [109]:
res_list

[{'region': 'Anhui',
  'word_list': [{'word': '女权', 'label': 1, 'value': 1.0, 'entropy': 1.0},
   {'word': '女性',
    'label': 1,
    'value': 0.6621848739495798,
    'entropy': 0.3300010253692607},
   {'word': '男人',
    'label': 1,
    'value': 0.47226890756302514,
    'entropy': 0.5317889711255445},
   {'word': '女人',
    'label': 1,
    'value': 0.45546218487394957,
    'entropy': 0.6292877563744558},
   {'word': '男性',
    'label': 1,
    'value': 0.3563025210084034,
    'entropy': 0.9613934983845343},
   {'word': '孩子',
    'label': 1,
    'value': 0.29747899159663865,
    'entropy': 0.022039539774381618},
   {'word': '田园',
    'label': 1,
    'value': 0.1764705882352941,
    'entropy': 0.2697969231999883},
   {'word': '社会',
    'label': 1,
    'value': 0.17478991596638654,
    'entropy': 0.33026727325225336},
   {'word': '微博',
    'label': 0,
    'value': 0.13789168771288657,
    'entropy': 0.29991078188686154},
   {'word': '男女',
    'label': 1,
    'value': 0.12100840336134455,
    

In [22]:
dict(sorted(jsd_shift.type2p_2.items(), key=lambda item: item[1], reverse=True))

{'女权': 0.031101673744691482,
 '女性': 0.025284607972591983,
 '男人': 0.014881695870953927,
 '女人': 0.013775382748652797,
 '男性': 0.011991006744941295,
 '孩子': 0.00881481745833482,
 '社会': 0.007976160736590415,
 '男权': 0.007833410656293494,
 '田园': 0.005478034331394311,
 '平等': 0.004907034010206631,
 '男女': 0.004675065129724136,
 '结婚': 0.0046036900895756756,
 '女生': 0.004139752328610685,
 '性别': 0.00401484600835088,
 '微博': 0.00401484600835088,
 '中国': 0.0036044395274972343,
 '权利': 0.003533064487348774,
 '平权': 0.003515220727311659,
 '支持': 0.0029977516862353237,
 '婚姻': 0.0029263766460868633,
 '喜欢': 0.0029263766460868633,
 '女拳': 0.0028906891260126336,
 '理解': 0.002837157845901288,
 '歧视': 0.002605188965418793,
 '群体': 0.002408907605010528,
 '这是': 0.0022840012847507225,
 '家庭': 0.0022483137646764927,
 '生育': 0.0021769387245280324,
 '生活': 0.0021055636843795725,
 '男生': 0.0020877199243424576,
 '思想': 0.0019271260840084223,
 '东西': 0.0018914385639341922,
 '选择': 0.0018735948038970773,
 '婚女': 0.0018557510438599622,
 '

In [30]:
jsd_shift.type2p_2['⌚️']

1.784376003711502e-05

In [29]:
comment_cut[(comment_cut['label']==1) & (comment_cut['location']=='广东')]['cut'].tolist()

["['结婚', '孩子', '结婚', '孩子', '婚驴']",
 "['此类', '平时', '少见', '这位', 'feminist', '女拳', '同学', '屏幕', '重拳', '出击']",
 "['夫妻', '相处', '抓拍', '小段', '原因', '好看', '人身', '攻击', '批评', '过头', '抠字眼', '估计', '微博', '那种', '文章', '家务', '平摊', '男生', '女生', '家务', '分担', '。。', '。。', '估计', '文章']",
 "['逻辑', '一辈子', '社会', '人文', '关怀', '残疾人', '社会', '关怀', '区别', '男女', '平等', '男女', '社会', '机会', '平等', '惩罚', '罪犯', '受害者', '实话', '男权', '附属品', '谢谢']",
 "['大叔', '阿姨', '网上', '粉丝', '巨多', '自称', '形容', '男性', '大V', '和菜头', '仲敬', '魅惑', '蓝心', '侮辱']",
 "['社会', '保守派', '特指', '女性', '偏坡', '既得利益', '自然', '同性恋', '支持', '女权', '运动', '90', 'gay', '仇女', '节奏', '两性', '矛盾', '源自于', '早期', '社会', '生产力', '性别', '对等', '社会', '女性', '脑力', '劳动', '丝毫', '男性', '生产力', '生产关系', '女性', '社会', '生产力', '地位', '历史', '男性', '地位', '主导', '些许', '矛盾', '矛盾', '小编', '弱智', '带两性', '矛盾', '节奏', '⌚️', 'cherry', '中国', '如出一辙', '带仇', '节奏', '男性', '嘴臭', '这不', '恶性', '循环', '缓和', '解决', '矛盾', '天天', '节奏']",
 "['我体', '体谅', '女友', '做好', '做好', '女权', '女权', '实处', '女生', '医生', '独立', '收入', '学历', '承认', '错误', '言论', '阻止', '

In [28]:
comment_cut

,Unnamed: 0,index,weibo_id,comment_text,gender,location,like,date,label,category,target,cut
0,0,0,J1zF7smOR,她们不结婚不要孩子，却觉得我们这种结婚有孩子的是婚驴,女,广东,14,12/05/2020 15:04,1,SCB,G,"['结婚', '孩子', '结婚', '孩子', '婚驴']"
1,1,1,GvXf0ymdJ,妈的什么时候有人来性侵我一下啊 不要封口费的那种,男,其他,36,23/08/2018 13:10,1,SO,I,"['妈的', '有人', '性侵', '封口费', '那种']"
2,2,2,J2XeR8rPn,我觉得这就是对一类人所有的特点进行形容吧，“爹味”也是贬义词，为什么不觉得有性别歧视意味呢？,女,浙江,0,21/05/2020 16:12,1,MA,G,"['这就', '一类', '形容', '爹味', '贬义词', '性别', '歧视', '意味']"
3,3,3,IxpfTCMAo,姐姐你不要急，关注久的都知道她骂的一切归根结底都是渴婚的，她并不骂学艺术的，她骂的是不肯像培...,女,江苏,1,31/03/2020 13:57,1,MA,I,"['姐姐', '关注', '渴婚', '骂学', '艺术', '培养', '儿子', '培养..."
4,4,4,J4B0Mps5Q,"给他们争取权利的不是女权是腐女，你该不会说腐女都女权吧？连女同性恋都不是全部女权，腐女？,男...",女,其他,0,##########,1,MA,G,"['权利', '女权', '腐女', '腐女', '女权', '同性恋', '女权', '腐..."
...,...,...,...,...,...,...,...,...,...,...,...,...
8964,8964,8964,GxUGteFvS,之前一直是五个男生那种，你们批评我吧,男,广东,0,05/09/2018 12:04,0,NaN,NaN,"['男生', '那种', '批评']"
8965,8965,8965,Fq0uZhLsx,中国的小孩也可以随母亲姓啊 我一个朋友就是随母亲姓的,男,其他,0,12/10/2017 11:50,0,NaN,NaN,"['中国', '小孩', '母亲', '朋友', '母亲']"
8966,8966,8966,H5OE5y1ij,所有为韦涛卖命而去攻击别人的炮灰都快点死.......咒语灵灵灵。。。。。。,男,广东,2,05/12/2018 17:50,0,NaN,NaN,"['韦涛', '卖命', '攻击', '炮灰', '.......', '咒语', '灵灵灵..."
8967,8967,8967,J2cWRaHx1,本质是阶级矛盾……比如你是个资本家，别人抱你大腿的时候就不会管你是男还是女,男,上海,1,16/05/2020 17:56,0,NaN,NaN,"['本质', '阶级', '矛盾', '资本家', '大腿']"
